In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
import pandas as pd

# path_to_files = "/content/drive/MyDrive/recommender_system_files/"
path_to_files = "./recommender_system_files/"
ratings = pd.read_csv(path_to_files + "ratings.csv", sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv(path_to_files + "movies.csv", sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [5]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [7]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [9]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
# mapping movie titles to indices
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
def genre_recommendations(title, n=10):
    index = indices[title]  # index of our movie
    sim_scores = list(enumerate(cosine_sim[index])) # similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1] # starting from 1 so that the movie itself doesn't come to number 1
    movie_indices = [i[0] for i in sim_scores]
    
    return movies[['title', 'genres']].iloc[movie_indices]

In [13]:
# Example
genre_recommendations('GoldenEye (1995)')

,title,genres
345,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
543,Surviving the Game (1994),"['Action', 'Adventure', 'Thriller']"
724,"Rock, The (1996)","['Action', 'Adventure', 'Thriller']"
788,Daylight (1996),"['Action', 'Adventure', 'Thriller']"
825,Chain Reaction (1996),"['Action', 'Adventure', 'Thriller']"
978,Maximum Risk (1996),"['Action', 'Adventure', 'Thriller']"
1467,Anaconda (1997),"['Action', 'Adventure', 'Thriller']"
1513,Con Air (1997),"['Action', 'Adventure', 'Thriller']"
1693,Firestorm (1998),"['Action', 'Adventure', 'Thriller']"
3686,"Perfect Storm, The (2000)","['Action', 'Adventure', 'Thriller']"



### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [89]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
data = pd.merge(ratings, movies, on='movie_id')

In [16]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,1,661,3,0,660,James and the Giant Peach (1996),"['Animation', ""Children's"", 'Musical']"
2,1,914,3,0,913,My Fair Lady (1964),"['Musical', 'Romance']"
3,1,3408,4,0,3407,Erin Brockovich (2000),['Drama']
4,1,2355,5,0,2354,"Bug's Life, A (1998)","['Animation', ""Children's"", 'Comedy']"


In [97]:
# data = data.assign(genres=data['genres'].str.split('|')).explode('genres')
# ensuring genres column is a list
data['genres'] = data['genres'].apply(lambda x: eval(x) if isinstance(x, str) else x)
# splitting genres column into separate rows
data = data.explode('genres')

In [98]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,0,660,James and the Giant Peach (1996),Animation
1,1,661,3,0,660,James and the Giant Peach (1996),Children's
1,1,661,3,0,660,James and the Giant Peach (1996),Musical
2,1,914,3,0,913,My Fair Lady (1964),Musical


In [99]:
total_user_movie_counts = data.groupby('user_id').size().reset_index(name='total_movies')
total_user_genres_counts = data.groupby(['user_id','genres']).size().reset_index(name='total_genres')

user_genre_df = pd.merge(total_user_movie_counts, total_user_genres_counts, on=['user_id'], how='left')
user_genre_df['genre_proportion'] = user_genre_df['total_genres'] / user_genre_df['total_movies']

user_genre_df.head(5)

,user_id,total_movies,genres,total_genres,genre_proportion
0,1,116,Action,5,0.043103
1,1,116,Adventure,5,0.043103
2,1,116,Animation,18,0.155172
3,1,116,Children's,20,0.172414
4,1,116,Comedy,14,0.120690


In [105]:
#and now that we have proportion of each genre to all movies
user_genre_matrix = user_genre_df.pivot(index='user_id', columns='genres', values='genre_proportion')
user_genre_matrix = user_genre_matrix.fillna(0)
user_genre_matrix.head(5)


genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user_id,,,,,,,,,,,,,,,,,,
1,0.043103,0.043103,0.155172,0.172414,0.120690,0.017241,0.000000,0.181034,0.025862,0.000000,0.000000,0.120690,0.000000,0.051724,0.025862,0.025862,0.017241,0.000000
2,0.194444,0.065972,0.000000,0.000000,0.086806,0.041667,0.000000,0.274306,0.003472,0.003472,0.006944,0.000000,0.010417,0.083333,0.059028,0.107639,0.052083,0.010417
3,0.186992,0.203252,0.024390,0.024390,0.243902,0.000000,0.000000,0.065041,0.016260,0.000000,0.024390,0.008130,0.008130,0.040650,0.048780,0.040650,0.016260,0.048780
4,0.327586,0.103448,0.000000,0.017241,0.000000,0.017241,0.000000,0.103448,0.034483,0.000000,0.051724,0.000000,0.000000,0.034483,0.155172,0.068966,0.051724,0.034483
5,0.088068,0.025568,0.011364,0.017045,0.159091,0.059659,0.017045,0.295455,0.000000,0.008523,0.028409,0.008523,0.022727,0.085227,0.042614,0.110795,0.017045,0.002841


In [109]:
def get_top_n_similar_users(user_id, n=5):
    user_preferences = user_genre_matrix.loc[user_id].values.reshape(1,-1)
    similarities = cosine_similarity(user_genre_matrix, user_preferences)
    similar_users = similarities.flatten().argsort()[::-1][1:n+1]   # excluding user itself
    return similar_users + 1

get_top_n_similar_users(1, 5)

array([4456, 1481, 4767, 5343, 4883], dtype=int64)

In [148]:

def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    #TODO
    similar_users = get_top_n_similar_users(user_id, n_similar_users)
    similar_users_data = data[data['user_id'].isin(similar_users)]
    target_user_data = data[data['user_id']==user_id]
    target_user_movies = set(target_user_data['movie_id'])

    similar_users_movies = similar_users_data[~similar_users_data['movie_id'].isin(target_user_movies)]

    all_recommended_movies = similar_users_movies.groupby('movie_id').size().reset_index(name='count')
    
    top_recommended_movies = all_recommended_movies.sort_values(by='count', ascending=False)[:n_recommendations]
    
    top_movies = pd.merge(top_recommended_movies, movies, on='movie_id', how='left')[['movie_id', 'title', 'genres']]

    return top_movies.sort_values(by="movie_id")


In [149]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)
recommended_movies


,movie_id,title,genres
8,34,Babe (1995),"[""Children's"", 'Comedy', 'Drama']"
9,912,Casablanca (1942),"['Drama', 'Romance', 'War']"
3,1032,Alice in Wonderland (1951),"['Animation', ""Children's"", 'Musical']"
7,1282,Fantasia (1940),"['Animation', ""Children's"", 'Musical']"
2,1617,L.A. Confidential (1997),"['Crime', 'Film-Noir', 'Mystery', 'Thriller']"
4,2078,"Jungle Book, The (1967)","['Animation', ""Children's"", 'Comedy', 'Musical']"
1,2080,Lady and the Tramp (1955),"['Animation', ""Children's"", 'Comedy', 'Musical..."
0,2081,"Little Mermaid, The (1989)","['Animation', ""Children's"", 'Comedy', 'Musical..."
5,2087,Peter Pan (1953),"['Animation', ""Children's"", 'Fantasy', 'Musical']"
6,2096,Sleeping Beauty (1959),"['Animation', ""Children's"", 'Musical']"


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [153]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [164]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
# Ratings.head()

In [165]:
# calculate rating matrix
ratings_matrix = Ratings.to_numpy()

In [166]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(ratings_matrix, k=50)
sigma = np.diag(sigma)

In [167]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)


In [176]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=Ratings.columns)
# predictions.head()

In [189]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == userID]

    # merge movies data with user_data
    user_full = user_data.merge(movies, on='movie_id', how='left').sort_values(['rating'], ascending=False)
    
    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = movies[~movies['movie_id'].isin(user_full['movie_id'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(),on='movie_id', how='left')
    recommendations = recommendations.rename(columns={user_row_number: "Predictions"}).sort_values(by='Predictions', ascending=False)
    
    return user_full, recommendations


In [190]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [191]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [192]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=40)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)

preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    try:
        user_row_number = user_id - 1  # User ID starts at 1, not 0
        prediction = preds_train.iloc[user_row_number, movie_id]
    except:
        prediction = np.nan
    return prediction

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.5301000025783873


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        #TODO
        pass

    def __len__(self):
        #TODO
        pass

    def __getitem__(self, idx):
        #TODO
        pass

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        #TODO
        pass

    def forward(self, user, movie):
        #TODO
        pass

# Create the dataset and dataloader


# Initialize the model, loss function and optimizer


# Training loop


# Evaluation


In [ ]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

In [ ]:
def predict_unrated_movies(user_id, model, ratings, movies):
    #TODO
    pass


# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                             title             genres  \
1235      3338            For All Mankind (1989)        Documentary   
227       1704          Good Will Hunting (1997)              Drama   
1792      1797                    Everest (1998)        Documentary   
114        318  Shawshank Redemption, The (1994)              Drama   
2638       669                  Aparajito (1956)              Drama   
100        920         Gone with the Wind (1939)  Drama|Romance|War   
3341       598            Window to Paris (1994)             Comedy   
497        428              Bronx Tale, A (1993)              Drama   
74        1198    Raiders of the Lost Ark (1981)   Action|Adventure   
3359      2503           Apple, The (Sib) (1998)              Drama   

      predicted_rating  
1235          4.692185  
227           4.664774  
1792          4.663356  
114           4.659920  
2638          4.656652  
100           4.645355  
3341          4.635198  
497           4.63

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

file_path = # TODO
df = pd.read_csv(file_path)


gmm = # TODO

log_likelihood = # TODO
bic = # TODO
aic = # TODO

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")